# Query -> Category Perfect Classification

<small>
(from <a href="http://maven.com/softwaredoug/cheat-at-search">Cheat at Search with Agents</a> training course by Doug Turnbull.)
</small>

The opposite of a baseline - what happens when query understanding goes perfectly? We get **perfect categorization** we see the impact of directly using the ground truth to evaluate to see the theoretical maximum.

1. Build a classifier that returns the ground truth for each query
2. Essentially filter (boost very highly) those category / subcategories if present


## Boilerplate

Install deps, mount GDrive, prompt for your OpenAI Key (placed in your GDrive), and import needed cheat at search helpers.

We cover this extensively in the [synonyms notebook](https://colab.research.google.com/drive/1aUCvcBa1YdmsbIgYc74jlknl9_iRotp1) walkthrough

In [ ]:
!pip install git+https://github.com/softwaredoug/cheat-at-search.git
from cheat_at_search.data_dir import mount
mount(use_gdrive=True)
from cheat_at_search.search import run_strategy, graded_bm25, ndcgs, ndcg_delta, vs_ideal
from cheat_at_search.wands_data import products, judgments

products

  Cloning https://github.com/softwaredoug/cheat-at-search.git to /tmp/pip-req-build-ycw3a6jx
  Running command git clone --filter=blob:none --quiet https://github.com/softwaredoug/cheat-at-search.git /tmp/pip-req-build-ycw3a6jx
  Resolved https://github.com/softwaredoug/cheat-at-search.git to commit 6a08d097f1d6eaa068fb61af47c621df1682f5e2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,...,title,description,category,sub_category,cat_subcat,product_name_snowball,product_description_snowball,subcategory,category_snowball,subcategory_snowball
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0,"[overallwidth-sidetoside:64.7, dsprimaryproduc...",...,solid wood platform bed,"good , deep sleep can be quite difficult to ha...",Furniture,Bedroom Furniture,Furniture / Bedroom Furniture,"Terms({'bed', 'wood', 'solid', 'platform'})","Terms({'style', 'decor', 'rest', 'natur', 'fee...",Bedroom Furniture,Terms({'furnitur'}),"Terms({'furnitur', 'bedroom'})"
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0,"[capacityquarts:7, producttype : slow cooker, ...",...,all-clad 7 qt . slow cooker,"create delicious slow-cooked meals , from tend...",Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances,"Terms({'cooker', 'all', 'clad', 'qt', 'slow', ...","Terms({'veggi', 'finish', 'aluminum', 'and', '...",Small Kitchen Appliances,"Terms({'kitchen', 'tabletop'})","Terms({'kitchen', 'small', 'applianc'})"
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0,"[features : keep warm setting, capacityquarts:...",...,all-clad electrics 6.5 qt . slow cooker,prepare home-cooked meals on any schedule with...,Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances,"Terms({'cooker', 'electr', 'qt', 'all', 'clad'...","Terms({'on', 'this', 'essenti', '26', 'timer',...",Small Kitchen Appliances,"Terms({'kitchen', 'tabletop'})","Terms({'kitchen', 'small', 'applianc'})"
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0,"[overallwidth-sidetoside:3.5, warrantylength :...",...,all-clad all professional tools pizza cutter,this original stainless tool was designed to c...,Browse By Brand,All-Clad,Browse By Brand / All-Clad,"Terms({'tool', 'cutter', 'pizza', 'all', 'clad...","Terms({'crust', 'this', 'pasta', 'was', 'desig...",All-Clad,"Terms({'brand', 'brows', 'by'})","Terms({'all', 'clad'})"
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0,"[compatibledoorthickness:1.375 '' , countryofo...",...,baldwin prestige alcott passage knob with roun...,the hardware has a rich heritage of delivering...,Home Improvement,Doors & Door Hardware,Home Improvement / Doors & Door Hardware,"Terms({'knob', 'baldwin', 'passag', 'prestig',...","Terms({'baldwin', 'style', 'on', 'hardwar', 'h...",Doors & Door Hardware,"Terms({'improv', 'home'})","Terms({'hardwar', 'door'})"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0,"[producttype : shower panel, spraypattern : ra...",...,malibu pressure balanced diverter fixed shower...,the malibu pressure balanced diverter fixed sh...,Home Improvement,Bathroom Remodel & Bathroom Fixtures,Home Improvement / Bathroom Remodel & Bathro...,"Terms(

## Create Cheating Classifier

Below we use the ground truth to classify our queries perfectly from our labeled data.

**The Goal** -- understand what MIGHT happen if our zero-shot LLM could always get the right answer. Then we can decide how much we want to invest here.

### Recreate Category Ground Truth

Our ground truth from previous notebooks mapping query -> relevant products -> the dominant categories for those products

In [ ]:
from cheat_at_search.wands_data import labeled_query_products, queries

def get_top_category(column, no_fit_label, cutoff=0.8):
    # Get relevant products per query
    top_products = labeled_query_products[labeled_query_products['grade'] == 2]

    # Aggregate top categories
    categories_per_query_ideal = top_products.groupby('query')[column].value_counts().reset_index()

    # Get as percentage of all categories for this query
    top_cat_proportion = categories_per_query_ideal.groupby(['query', column]).sum() / categories_per_query_ideal.groupby('query').sum()
    top_cat_proportion = top_cat_proportion.drop(columns=column).reset_index()

    # Only look at cases where the category is > 0.8
    top_cat_proportion = top_cat_proportion[top_cat_proportion['count'] > cutoff]
    top_cat_proportion[column].fillna(no_fit_label, inplace=True)
    ground_truth_cat = top_cat_proportion
    # Give No Category Fits to all others without dominant category
    ground_truth_cat = ground_truth_cat.merge(queries, how='right', on='query')[['query', column, 'count']]
    ground_truth_cat[column].fillna(no_fit_label, inplace=True)
    return ground_truth_cat

def get_pred(cat, column):
    if column == 'category':
        return cat.category
    elif column == 'sub_category':
        return cat.sub_category
    else:
        raise ValueError(f"Unknown column {column}")


def prec_cat(ground_truth, column, no_fit_label, categorized, N=500):
    hits = []
    misses = []
    for _, row in ground_truth.sample(frac=1).iterrows():
        query = row['query']
        expected_category = row[column]

        cat = categorized(query)
        pred = get_pred(cat, column)
        if pred == no_fit_label:
            print(f"Skipping {query}")
            continue
        if pred == expected_category.strip():
            hits.append((expected_category, cat))
        else:
            print("***")
            print(f"{query} -- predicted:{cat.category} != expected:{expected_category.strip()}")
            misses.append((expected_category, cat))
            num_so_far = len(hits) + len(misses)
            print(f"prec (N={num_so_far}) -- {len(hits) / (len(hits) + len(misses))}")
            print(f"coverage {num_so_far / len(ground_truth)}")

        if len(hits) + len(misses) > N:
            break
    return len(hits) / (len(hits) + len(misses)), num_so_far / len(ground_truth)

In [ ]:
ground_truth_cat = get_top_category('category', 'No Category Fits')
ground_truth_cat
ground_truth_sub_cat = get_top_category('sub_category', 'No SubCategory Fits')
ground_truth_sub_cat

/tmp/ipython-input-2945477783.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  top_cat_proportion[column].fillna(no_fit_label, inplace=True)
/tmp/ipython-input-2945477783.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

,query,sub_category,count
0,salon chair,No SubCategory Fits,NaN
1,smart coffee table,Living Room Furniture,1.000000
2,dinosaur,No SubCategory Fits,NaN
3,turquoise pillows,Decorative Pillows & Blankets,0.963636
4,chair and a half recliner,Living Room Furniture,0.956522
...,...,...,...
475,rustic twig,No SubCategory Fits,NaN
476,nespresso vertuo next premium by breville with...,No SubCategory Fits,NaN
477,pedistole sink,No SubCategory Fits,NaN
478,54 in bench cushion,No SubCategory Fits,NaN


## Query -> Category classification  structure

As taken from previous notebooks, our category + subcategory classifications.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal
from cheat_at_search.enrich import AutoEnricher


Categories = Literal['Furniture',
                     'Home Improvement',
                     'Décor & Pillows',
                     'Outdoor',
                     'Storage & Organization',
                     'Lighting',
                     'Rugs',
                     'Bed & Bath',
                     'Kitchen & Tabletop',
                     'Baby & Kids',
                     'School Furniture and Supplies',
                     'Appliances',
                     'Holiday Décor',
                     'Commercial Business Furniture',
                     'Pet',
                     'Contractor',
                     'Sale',
                     'Foodservice ',
                     'Reception Area',
                     'Clips',
                     'No Category Fits']

SubCategories = Literal['Bedroom Furniture',
 'Small Kitchen Appliances',
 'All-Clad',
 'Doors & Door Hardware',
 'Bathroom Remodel & Bathroom Fixtures',
 'Home Accessories',
 'Living Room Furniture',
 'Outdoor Décor',
 'Flooring, Walls & Ceiling',
 'Garage & Outdoor Storage & Organization',
 'Cookware & Bakeware',
 'Bedding',
 'Kitchen Utensils & Tools',
 'Shower Curtains & Accessories',
 'Wall Shelving & Organization',
 'Clocks',
 'Bedding Essentials',
 'Kitchen & Dining Furniture',
 'Office Furniture',
 'Tableware & Drinkware',
 'Nursery Bedding',
 'Cat',
 'Outdoor Shades',
 'Outdoor & Patio Furniture',
 'Ceiling Lights',
 'Area Rugs',
 'Outdoor Lighting',
 'Window Treatments',
 'Garden',
 'Closet Storage & Organization',
 'Wall Décor',
 'Mirrors',
 'Shoe Storage',
 'Toddler & Kids Playroom',
 'Game Tables & Game Room Furniture',
 'Decorative Pillows & Blankets',
 'School Furniture',
 'Wall Lights',
 'Bathroom Storage & Organization',
 'Commercial Office Furniture',
 'Flowers & Plants',
 'Mattresses & Foundations',
 'Area Rugs',
 'Cleaning & Laundry Organization',
 'Kitchen Organization',
 'Candles & Holders',
 'Christmas',
 'Toddler & Kids Bedroom Furniture',
 'Front Door Décor & Curb Appeal',
 'Storage Furniture',
 'School Spaces',
 'Hardware',
 'Light Bulbs & Hardware',
 'Ceiling Fans',
 'Doormats',
 'Entry & Hallway',
 'Storage Containers & Drawers',
 'Holiday Lighting',
 'Kitchen Mats',
 'Facilities & Maintenance',
 'Table & Floor Lamps',
 'Bird',
 'Kitchen Appliances',
 'Building Equipment',
 'Art',
 'Picture Frames & Albums',
 'Outdoor Heating',
 'Outdoor Recreation',
 'Bathroom Accessories & Organization',
 'School Boards & Technology',
 'Closeout',
 'Reception Seating',
 'Foodservice Tables',
 'Kitchen Remodel & Kitchen Fixtures',
 'Hot Tubs & Saunas',
 'Teen Bedroom Furniture',
 'Outdoor Fencing & Flooring',
 'Chairs',
 'Bath Rugs & Towels',
 'Fish',
 'Dog',
 'Chicken',
 'Boards & Tech Accessories',
 'Commercial Contractor',
 'Clamps',
 'Jewelry Organization',
 'Entry & Mudroom Furniture',
 'Outdoor Cooking & Tableware',
 'Seasonal Décor',
 'Nursery Furniture',
 'Storage & Organization Sale',
 'Washers & Dryers',
 'Baby & Kids Décor & Lighting',
 'Outdoor Remodel',
 'Plumbing',
 'Birch Lane™',
 'Office Organization',
 'Kitchen & Dining Sale',
 'Holiday Lighting',
 'Baby & Kids Storage',
 'Shop All Characters',
 'Commercial Kitchen',
 'Guest Room Amenities',
 'Charlton Home',
 'Wade Logan®',
 'Heating, Cooling & Air Quality',
 'Thanksgiving',
 'Fourth of July',
 'Vacuums & Deep Cleaners',
 'Stair Tread Rugs',
 'Small Spaces',
 'Toddler & Kids Bedding & Bath',
 'Classroom Décor',
 'Early Education Play Area',
 'Zoomie Kids',
 'Fryers',
 'August Grove',
 'Dorm Décor & Back to School Essentials',
 'Symple Stuff',
 'Wayfair Basics®',
 'The Holiday Aisle',
 'Chair Pads & Cushions',
 'The Monogram Shop',
 'Wedding',
 'Wedding',
 'Reception Desks & Tables',
 'Rug Pads',
 'Latitude Run',
 'Accommodations Furniture',
 'Easter',
 'Furniture Sale',
 'Shop All Characters',
 'Novelty Lights',
 "Valentine's Day",
 'Outdoor Sale',
 'Classroom & Training Furniture',
 'Rebrilliant',
 'Rug Pads',
 'Commercial Kitchen Storage',
 'Teen Bedding',
 'Tommy Bahama Home',
 'Appliances Sale',
 'Massage Products',
 'No SubCategory Fits']

class Query(BaseModel):
    """
    Base model for search queries, containing common query attributes.
    """
    keywords: str = Field(
        ...,
        description="The original search query keywords sent in as input"
    )

class QueryCategory(Query):
    """
    Structured representation of a search query for furniture e-commerce.
    Inherits keywords from the base Query model and adds category and sub-category.
    """
    category: Categories = Field(
        description="Category of the product"
    )
    sub_category: SubCategories = Field(
        description="Sub-category of the product"
    )

    @property
    def classification(self) -> str:
        return f"{self.category} / {self.sub_category}"





### Query classification code (cheating)

We lookup in the ground truth the category and subcategory and return the right answer

In [ ]:
from re import sub

def categorized(query):
    category = "No Category Fits"
    sub_category = "No SubCategory Fits"
    if query in ground_truth_cat['query'].values:
        cat_at_query = ground_truth_cat[ground_truth_cat['query'] == query]['category']
        cat_at_query = cat_at_query.values[0].strip()
        category = cat_at_query if cat_at_query else "No Category Fits"
    if query in ground_truth_sub_cat['query'].values:
        sub_cat_at_query = ground_truth_sub_cat[ground_truth_sub_cat['query'] == query]['sub_category']
        sub_cat_at_query = sub_cat_at_query.values[0].strip()
        sub_category = sub_cat_at_query if sub_cat_at_query else "No SubCategory Fits"
        print(sub_category)
    return QueryCategory(keywords=query, category=category, sub_category=sub_category)


categorized("tv stand")

QueryCategory(keywords='tv stand', category='No Category Fits', sub_category='No SubCategory Fits')

In [ ]:
TEST_QUERY = "island estate coffee table"
ground_truth_cat[ground_truth_cat['query'] == TEST_QUERY], categorized(TEST_QUERY)

Living Room Furniture


(                          query    category  count
 370  island estate coffee table  Furniture     1.0,
 QueryCategory(keywords='island estate coffee table', category='Furniture', sub_category='Living Room Furniture'))

### Category search strategy

The same search strategy from before, but here we pass the cheating classifier

In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer
from cheat_at_search.strategy.strategy import SearchStrategy
import numpy as np


class CategorySearch(SearchStrategy):
    def __init__(self, products, query_to_cat,
                 name_boost=9.3,
                 description_boost=4.1,
                 category_boost=100,
                 sub_category_boost=50):
        super().__init__(products)
        self.index = products
        self.index['product_name_snowball'] = SearchArray.index(
            products['product_name'], snowball_tokenizer)
        self.index['product_description_snowball'] = SearchArray.index(
            products['product_description'], snowball_tokenizer)

        cat_split = products['category hierarchy'].fillna('').str.split("/")

        products['category'] = cat_split.apply(
            lambda x: x[0].strip() if len(x) > 0 else ""
        )
        products['subcategory'] = cat_split.apply(
            lambda x: x[1].strip() if len(x) > 1 else ""
        )
        self.index['category_snowball'] = SearchArray.index(
            products['category'], snowball_tokenizer
        )
        self.index['subcategory_snowball'] = SearchArray.index(
            products['subcategory'], snowball_tokenizer
        )

        self.query_to_cat = query_to_cat
        self.name_boost = name_boost
        self.description_boost = description_boost
        self.category_boost = category_boost
        self.sub_category_boost = sub_category_boost

    def search(self, query, k=10):
        """Dumb baseline lexical search, but add a constant boost when
           the desired category or subcategory"""
        bm25_scores = np.zeros(len(self.index))
        structured = self.query_to_cat(query)
        tokenized = snowball_tokenizer(query)
        print(query, structured)

        # ****
        # Baseline BM25 search from before
        for token in tokenized:
            bm25_scores += self.index['product_name_snowball'].array.score(token) * self.name_boost
            bm25_scores += self.index['product_description_snowball'].array.score(
                token) * self.description_boost

        # ****
        # If there's a subcategory, boost that by a constant amount
        if structured.sub_category and structured.sub_category != "No SubCategory Fits":
            tokenized_subcategory = snowball_tokenizer(structured.sub_category)
            subcategory_match = np.ones(len(self.index))
            if tokenized_subcategory:
                subcategory_match = self.index['subcategory_snowball'].array.score(tokenized_subcategory) > 0
            bm25_scores[subcategory_match] += self.sub_category_boost

        # ****
        # If there's a category, boost that by a constant amount
        if structured.category and structured.category != "No Category Fits":
            tokenized_category = snowball_tokenizer(structured.category)
            category_match = np.ones(len(self.index))
            if tokenized_category:
                category_match = self.index['category_snowball'].array.score(tokenized_category) > 0
            bm25_scores[category_match] += self.category_boost

        top_k = np.argsort(-bm25_scores)[:k]
        scores = bm25_scores[top_k]

        return top_k, scores


In [ ]:
categorized_search = CategorySearch(products, categorized)
graded_categorized = run_strategy(categorized_search, judgments)
graded_categorized

2026-02-08 23:05:57,420 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2026-02-08 23:05:57,431 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2026-02-08 23:05:57,434 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

No SubCategory Fits
elephant framed keywords='elephant framed' category='No Category Fits' sub_category='No SubCategory Fits'
Office Furniture
adjustable height artist stool keywords='adjustable height artist stool' category='Furniture' sub_category='Office Furniture'
Wall Décor
window wall accent keywords='window wall accent' category='Décor & Pillows' sub_category='Wall Décor'
No SubCategory Fits
chair pillow cushion keywords='chair pillow cushion' category='No Category Fits' sub_category='No SubCategory Fits'
No SubCategory Fits
48 inch bathroom vanity with trough sink keywords='48 inch bathroom vanity with trough sink' category='No Category Fits' sub_category='No SubCategory Fits'
No SubCategory Fits
rattan truck keywords='rattan truck' category='No Category Fits' sub_category='No SubCategory Fits'
No SubCategory Fits
hardwood beds keywords='hardwood beds' category='No Category Fits' sub_category='No SubCategory Fits'
Office Furniture
worn leather office chair keywords='worn leathe

Searching:  93%|█████████▎| 445/480 [00:14<00:01, 22.69it/s]

Kitchen Appliances
48 inch kitchen hood keywords='48 inch kitchen hood' category='Appliances' sub_category='Kitchen Appliances'
No SubCategory Fits
rustic twig keywords='rustic twig' category='Décor & Pillows' sub_category='No SubCategory Fits'
No SubCategory Fits
nespresso vertuo next premium by breville with aeroccino keywords='nespresso vertuo next premium by breville with aeroccino' category='No Category Fits' sub_category='No SubCategory Fits'
No SubCategory Fits
pedistole sink keywords='pedistole sink' category='No Category Fits' sub_category='No SubCategory Fits'
No SubCategory Fits
54 in bench cushion keywords='54 in bench cushion' category='No Category Fits' sub_category='No SubCategory Fits'
No SubCategory Fits
rack glass keywords='rack glass' category='No Category Fits' sub_category='No SubCategory Fits'


Searching: 100%|██████████| 480/480 [00:15<00:00, 30.14it/s]


,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,...,subcategory,score,query,query_id,rank,grade,discounted_gain,idcg,dcg,ndcg
0,7465,hair salon chair,Massage Chairs|Recliners,Furniture / Living Room Furniture / Chairs & S...,offers a wide selection of professional salon ...,fauxleathertype : pu|legheight-toptobottom:18|...,69.0,4.5,53.0,"[fauxleathertype : pu, legheight-toptobottom:1...",...,Living Room Furniture,76.009592,salon chair,0,1,2.0,3.00,8.786905,8.10119,0.921962
1,25431,barberpub salon massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,salon chairs are a wonderful avenue for hairst...,supplierintendedandapproveduse : non residenti...,4.0,5.0,4.0,[supplierintendedandapproveduse : non resident...,...,Living Room Furniture,67.237749,salon chair,0,2,2.0,1.50,8.786905,8.10119,0.921962
2,7468,mercer41 hair salon chair hydraulic styling ch...,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,mercer41 beauty offers a wide selection profes...,seatfillmaterial : foam|waterrepellant : no re...,1.0,5.0,1.0,"[seatfillmaterial : foam, waterrepellant : no ...",...,Living Room Furniture,63.749867,salon chair,0,3,2.0,1.00,8.786905,8.10119,0.921962
3,39461,professional salon reclining massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,new and in a good condition . first-rate metal...,overalldepth-fronttoback:39.4|warrantylength:1...,NaN,NaN,NaN,"[overalldepth-fronttoback:39.4, warrantylength...",...,Living Room Furniture,58.923357,salon chair,0,4,2.0,0.75,8.786905,8.10119,0.921962
4,9234,beauty salon task chair,NaN,Furniture / Office Furniture / Office Chairs,"applicable scene : office , home life , beauty...",overallheight-toptobottom:37|backcolor : brown...,NaN,NaN,NaN,"[overallheight-toptobottom:37, backcolor : bro...",...,Office Furniture,57.375283,salon chair,0,5,1.0,0.20,8.786905,8.10119,0.921962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,22194,wine glass rack,Kitchen Sink Storage,Kitchen & Tabletop / Kitchen Organization / Co...,drip-dry up to eight wineglasses with this cle...,glasscapacity:8|countryoforigin : united state...,5.0,4.5,3.0,"[glasscapacity:8, countryoforigin : united sta...",...,Kitchen Organization,59.272195,rack glass,487,6,0.0,0.00,8.786905,0.00000,0.000000
4796,40243,madisen hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,complement your farmhouse kitchen decor with t...,producttype : wine glass rack|overallwidth-sid...,29.0,5.0,20.0,"[producttype : wine glass rack, overallwidth-s...",...,Tableware & Drinkware,58.455256,rack glass,487,7,0.0,0.00,8.786905,0.00000,0.000000
4797,40244,kena hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,spruce up your farmhouse kitchen decor with th...,warrantylength:1 year|producttype : wine glass...,23.0,5.0,18.0,"[warrantylength:1 year, producttype : wine gla...",...,Tableware & Drinkware,58.261101,rack glass,487,8,0.0,0.00,8.786905,0.00000,0.000000
4798,39976,wall mounted wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,"the latest addition to this collection , this ...",overallheight-toptobottom:4|design : wall moun...,34.0,4.5,18.0,"[overallheight-toptobottom:4, design : wall mo...",...,Tableware & Drinkware,57.824246,rack glass,487,9,0.0,0.00,8.786905,0.00000,0.000000


### Analyze the results

What's the upside / downside?

In [ ]:
ndcgs(graded_bm25).mean(), ndcgs(graded_categorized).mean()

(np.float64(0.5411098691836396), np.float64(0.5748049609357359))

In [ ]:
deltas = ndcg_delta(graded_categorized, graded_bm25)
deltas

,ndcg
query,
drum picture,0.886510
bathroom freestanding cabinet,0.692589
outdoor lounge chair,0.658131
wood rack wide,0.591790
outdoor light fixtures,0.550061
...,...
wall shelves,-0.099264
fleur de lis living candle wall sconce bronze,-0.106219
zodiac pillow,-0.113806


In [ ]:
sig_improved = len(deltas[deltas > 0.1])
print(f"Num Significatly Improved: {sig_improved}")
deltas[deltas > 0.1]

Num Significatly Improved: 56


,ndcg
query,
drum picture,0.886510
bathroom freestanding cabinet,0.692589
outdoor lounge chair,0.658131
wood rack wide,0.591790
outdoor light fixtures,0.550061
bathroom vanity knobs,0.524455
door jewelry organizer,0.495958
beds that have leds,0.493565
non slip shower floor tile,0.477081


In [ ]:
sig_harmed = len(deltas[deltas < -0.1])
print(f"Num Significatly Harmed: {sig_harmed}")
print(f"Prop improved/harmed: {sig_improved / (sig_harmed + sig_improved)} | {sig_harmed / (sig_harmed + sig_improved)}")
deltas[deltas < -0.1]

Num Significatly Harmed: 4
Prop improved/harmed: 0.9333333333333333 | 0.06666666666666667


,ndcg
query,
fleur de lis living candle wall sconce bronze,-0.106219
zodiac pillow,-0.113806
papasan chair frame only,-0.123290
bed side table,-0.333333


In [ ]:
impacted_queries = deltas[deltas > 0.1].index.to_list() + deltas[deltas < -0.1].index.to_list()
impacted_queries

['drum picture',
 'bathroom freestanding cabinet',
 'outdoor lounge chair',
 'wood rack wide',
 'outdoor light fixtures',
 'bathroom vanity knobs',
 'door jewelry organizer',
 'beds that have leds',
 'non slip shower floor tile',
 'turquoise chair',
 'modern outdoor furniture',
 'podium with locking cabinet',
 'closet storage with zipper',
 'barstool patio sets',
 'ayesha curry kitchen',
 'led 60',
 'wisdom stone river 3-3/4',
 'liberty hardware francisco',
 'french molding',
 'glass doors for bath',
 'accent leather chair',
 'dark gray dresser',
 'wainscoting ideas',
 'floating bed',
 'dining table vinyl cloth',
 'entrance table',
 'storage dresser',
 'almost heaven sauna',
 'toddler couch fold out',
 'outdoor welcome rug',
 'wooden chair outdoor',
 'emma headboard',
 'outdoor privacy wall',
 'driftwood mirror',
 'white abstract',
 'bedroom accessories',
 'bathroom lighting',
 'light and navy blue decorative pillow',
 'gnome fairy garden',
 'medium size chandelier',
 'above toilet cab

### Look at a query

In [ ]:
QUERY = "zodiac pillow"
categorized(QUERY)

Decorative Pillows & Blankets


QueryCategory(keywords='zodiac pillow', category='Décor & Pillows', sub_category='Decorative Pillows & Blankets')

In [ ]:
graded_categorized[graded_categorized['query'] == QUERY][['product_name', 'category hierarchy', 'grade']]

,product_name,category hierarchy,grade
1200,leo zodiac sign pillow cover,Décor & Pillows / Decorative Pillows & Blanket...,1.0
1201,gemini zodiac sign pillow cover,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1202,capricorn zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1203,cancer zodiac sign pillow cover,Décor & Pillows / Decorative Pillows & Blanket...,1.0
1204,dulin aries zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1205,dulin leo zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1206,dulin scorpio zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1207,dulin cancer zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1208,dulin libra zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1209,dulin sagittarius zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0


In [ ]:
graded_bm25[graded_bm25['query'] == QUERY][['product_name', 'category hierarchy', 'grade']]

,product_name,category hierarchy,grade
1200,gemini zodiac sign pillow cover,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1201,cancer zodiac sign pillow cover,Décor & Pillows / Decorative Pillows & Blanket...,1.0
1202,capricorn zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1203,leo zodiac sign pillow cover,Décor & Pillows / Decorative Pillows & Blanket...,1.0
1204,dulin pisces zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1205,dulin cancer zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1206,dulin gemini zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1207,dulin aries zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1208,dulin libra zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0
1209,dulin leo zodiac sign throw pillow,Décor & Pillows / Decorative Pillows & Blanket...,2.0


## Discussion points / questions

* Why do we still have a few negative 'deltas'?
* Is this truly the "upper bound" of having a category / subcategory in our ranking function?